# Demonstration of the full holography wofklow

This notebook demonstrates the full holography workflow, from the raw data to the final .nxs file.  

## Importing libraries

In [ ]:
from align_panel.data_structure import ImageSetHolo
import matplotlib
matplotlib.use('TkAgg') # solves problem with interactive matplotlib windows
import matplotlib.pyplot as plt

## Loading data

Now we will create two instances of the class `ImageSetHolo` and load the data.
Insert the images into chosen folder and copy relative paths into the `path1`, `path2`, `path3`, `path4` variables.

example: 
<br>
path1 = 'data/Hb-.dm3'
<br>
path2 = 'data/Rb-.dm3'
<br>
path3 = 'data/Hb+.dm3'
<br>
path4 = 'data/Rb+.dm3'

In [ ]:
path1 = '/data/Hb-.dm3'
path2 = '/data/Rb-.dm3'
path3 = '/data/Hb+.dm3'
path4 = '/data/Rb+.dm3'

In [ ]:
image_set1 = ImageSetHolo.load(path1, path2) # reference imageset
image_set2 = ImageSetHolo.load(path3, path4) # imageset to be aligned

## Check the loaded data

With the `Hyperspy` functions `plot` and `axes_manager` we can check the loaded data. Also, to see the metadata, call the `metadata` attribute of the `ImageSetHolo` class.
Same can be done for the second instance of the class.

In [ ]:
image_set1.image.axes_manager

In [ ]:
image_set1.image.plot()
plt.show()

In [ ]:
image_set1.image.metadata

## Calculation of the phase shift
For the phase calculation, call the `phase_calculation` method. It has several paramaters definig the calculation and also the plotting of the results. It is also possible to save unwrapped phase shift as a .png file outside the .nxs file.

In [ ]:
image_set1.phase_calculation()
image_set2.phase_calculation()

## Alignments

Now, to gain the final information and the contribution of magnetic induction to the phase shift, we need to align the images. For this, several methods can be called. In this notebook, we will use `PointAlignments`. 
<br>
The call for CropAlignments is:
<br>
`from align_panel.align.crop import CropAlignments`
<br>
The call for FineAlignments is:
<br>
`from align_panel.align.fine import FineAlignments`
<br>
<br>
All the alignment techniques have numpy arrays as an input for the images, therefore we need to obtain the numpy arrays from the `ImageSetHolo` instances.

In [ ]:
from align_panel.align.points import PointAlignments

In [ ]:
image_1 = image_set1.unwrapped_phase.data
image_2 = image_set2.unwrapped_phase.data

In [ ]:
result = PointAlignments(image_1, image_2,8)

The result is an instance of the `PointAlignments` class. To see the results, call `result.tmat` or `plt.imshow(result.result_image,cmap= 'gray')` to vizualize the transformation matrix or the result image. 

In [ ]:
result.tmat

In [ ]:
plt.imshow(result.result_image,cmap= 'gray')
plt.show()

Now, let's save the transformation matrix into second imageset. We are calling the `params` attribute to save only the transformation matrix as a numpy array, not as a `AffineTransform` object.

In [ ]:
image_set2.tmat = result.tmat.params

Now, both image sets can be saved within one .nxs file with the `save` method and will contain all the information about the phase calculation and the alignment.

In [ ]:
image_set1.save('test.nxs')
image_set2.save('test.nxs')

Let's check what we saved.

In [ ]:
image_set1.show_content('test.nxs',scope = 'full')

Let's check, if all the calculation parameters and transformation matrix saved correctly. We can load the 1. imageset from the .nxs file and check the `image.metadata` and `tmat` attributes.

In [ ]:
image_set_loaded = ImageSetHolo.load_from_nxs('test.nxs',id_number=1)

In [ ]:
image_set_loaded.tmat 

In [ ]:
image_set_loaded.image.metadata